In [1]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from loader import MNIST

# the functions reads the data and returns the test and train set along with its labels
def load_dataset():
    mndata = MNIST('./data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train = X_train/255.0
    X_test = X_test/255.0
    return (X_train, labels_train), (X_test, labels_test)

(X_train, labels_train), (X_test, labels_test) = load_dataset()

# converts each coordinate in {0,1,...,9} to a 10 bit indicator-vector 
def one_shot(labels,k=10):
    label_vectors = np.zeros((len(labels),k))
    for i in range(len(labels)):
        label_vectors[i][labels[i]] = 1
    return label_vectors

def train(X,Y,lmd):
    d = len(X[0])
    W = linalg.solve((np.matmul(np.transpose(X),X)+lmd*np.identity(d)),np.matmul(np.transpose(X),Y))
    return W

# computes WX and classifies by the index with the maximum value
def predict(W,X):
    pred = np.transpose(np.matmul(np.transpose(W),np.transpose(X)))
    return np.argmax(pred,axis=1)

# returns a random gaussian matrix of dimension n by k with mean mu and variance var
def random_gaussian_matrix(mu,var,n,k):
    X = np.sqrt(var) * np.random.randn(n,k) + mu
    return X

# returns a random uniform vector of dimension size with each coordinate in [range_left,range_right]
def random_uniform_vector(range_left, range_right,size):
    X = np.random.uniform(range_left,range_right,size)
    return X

# Part (b)
Y = one_shot(labels_train,10)

# Part(c)
W_opt = train(X_train,Y,0.0001)
pred_train = predict(W_opt,X_train)
pred_test = predict(W_opt,X_test)
train_accuracy = sum(pred_train == labels_train)/len(labels_train)
test_accuracy = sum(pred_test == labels_test)/len(labels_test)
print('Train accuracy =',train_accuracy,', and test accuracy =',test_accuracy)

# Part (d)

# Cross Validation Step
#permutation = np.random.permutation(len(X_train))
#new_train_size = int(0.8*len(X_train))
#perm1 = permutation[:new_train_size]
#perm2 = permutation[new_train_size:]
#X_train_new = X_train[perm1]
#X_test_new = X_train[perm2]
#labels_train_new = labels_train[perm1]
#labels_test_new = labels_train[perm2]

#p = [10,50,100,500,1000,2000,3000,4000,5000,6000]
#classification_error = np.zeros(10)
#validation_error = np.zeros(10)

#for i in range(10):
#    G = random_gaussian_matrix(0,np.sqrt(0.1),p[i],len(X_train_new[0]))
#    b = random_uniform_vector(0,2*np.pi,p[i])
#    X_trn = np.cos(np.matmul(X_train_new,np.transpose(G)) + np.tile(b,(len(X_train_new),1)))
#    X_tst = np.cos(np.matmul(X_test_new,np.transpose(G)) + np.tile(b,(len(X_test_new),1)))
#    Y = one_shot(labels_train_new,10)
#    W_opt = train(X_trn,Y,0.0001)
#    pred_train = predict(W_opt,X_trn)
#    pred_test = predict(W_opt,X_tst)
#    train_accuracy = sum(pred_train == labels_train_new)/len(labels_train_new)
#    test_accuracy = sum(pred_test == labels_test_new)/len(labels_test_new)
#    classification_error[i]=1-train_accuracy
#    validation_error[i]=1-test_accuracy

#print(classification_error,validation_error)

#plt.plot(p,classification_error,label="classification error")
#plt.plot(p,validation_error,label="validation error")
#plt.show()

# Part (e)

G = random_gaussian_matrix(0,0.1,6000,len(X_train[0]))
b = random_uniform_vector(0,2*np.pi,6000)

# performing the feature transofrmation
X_trn = np.transpose(np.cos(np.matmul(G,np.transpose(X_train)) + np.transpose(np.tile(b,(len(X_train),1)))))
X_tst = np.transpose(np.cos(np.matmul(G,np.transpose(X_test)) + np.transpose(np.tile(b,(len(X_test),1)))))

Y = one_shot(labels_train,10)
W_opt = train(X_trn,Y,0.0001)
pred_train = predict(W_opt,X_trn)
pred_test = predict(W_opt,X_tst)
train_accuracy = sum(pred_train == labels_train)/len(labels_train)
test_accuracy = sum(pred_test == labels_test)/len(labels_test)
print('Test accuracy (with feature transformation) =',test_accuracy)

Train accuracy = 0.85195 , and test accuracy = 0.8534
Test accuracy (with feature transformation) = 0.9494
